In [59]:
#import the necessary data
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.22.0         | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environ

In [62]:
#Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
text= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
data = BeautifulSoup(web_text, 'lxml')

In [68]:
#create the dataframe
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # remove the new line char from neighborhood cell
df = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
df = pd.DataFrame.from_dict(dict(toronto_neighorhood))
df.head()



,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [70]:
#drop Not assigned
df = df.dropna()
df = df[(df.PostalCode != 'Not assigned\n' ) & (df.Borough != 'Not assigned\n') & (df.Neighborhood != 'Not assigned\n')]
df = df.replace('\n','', regex=True)
df.head(20)

,PostalCode,Borough,Neighborhood
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"
8,M9A\n,Etobicoke\n,Islington Avenue
9,M1B\n,Scarborough\n,"Malvern, Rouge"
11,M3B\n,North York\n,Don Mills
12,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
13,M5B\n,Downtown Toronto\n,"Garden District, Ryerson"


In [73]:
df.reset_index(drop=True, inplace= True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [74]:
#group Neighborhoods 
df_grouped = df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [57]:
geo= pd.read_csv('http://cocl.us/Geospatial_data')  
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [76]:
df2 = df.set_index('PostalCode')
geo2 = geo.set_index('Postal Code')
df_geo= pd.concat([df2, geo2], axis=1, join='inner')

df_geo.index.name = 'PostalCode'
df_geo.reset_index(inplace=True)

df_geo.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
